# Optimization of the Object Detection Operator with YOLO

[Ultralytics YOLOv8](https://github.com/ultralytics/ultralytics) is a cutting-edge, state-of-the-art (SOTA) model that builds upon the success of previous YOLO versions and introduces new features and improvements to further boost performance and flexibility. YOLOv8 is designed to be fast, accurate, and easy to use, making it an excellent choice for a wide range of object detection and tracking, instance segmentation, image classification and pose estimation tasks.

## Run with YOLO

First, let's run YOLO prediction code, which will be very simple according to the [tutorial](https://docs.ultralytics.com/modes/predict/).

In [ ]:
from ultralytics import YOLO
model = YOLO("yolov8n.pt") 
result = model("https://ultralytics.com/images/bus.jpg")[0]

Then, we can get the information we want based on the predicted `result`, such as `boxes`, `classes` and `scores`.

In [ ]:
boxes = [list(map(int, xyxy)) for xyxy in result.boxes.xyxy]
classes = [result.names[int(i)] for i in result.boxes.cls]
scores = result.boxes.conf.tolist()
print(boxes, classes, scores)

## Develop yolo operator

According to [yolov5 operator](https://towhee.io/object-detection/yolov5), we can run the pipeline for image object  detection in the following code:

In [ ]:
from towhee import pipe, ops, DataCollection

p = (
    pipe.input('path')
        .map('path', 'img', ops.image_decode.cv2_rgb())
        .map('img', ('box', 'class', 'score'), ops.object_detection.yolov5())
        .map(('img', 'box'), 'object', ops.image_crop(clamp=True))
        .output('img', 'object', 'class')
)

In [ ]:
DataCollection(p("https://ultralytics.com/images/bus.jpg")).show()

Next, we can optimize the yolo arithmetic so as to support the yolov8 model, so we can develop yolov8 operator based on the yolov5 operator, for example, we develop the YOLOv8 class and develop the `__init__` and `__call__` function, so as to support YOLOv8 model.

In [ ]:
import numpy
from towhee import register
from towhee.operator import NNOperator
from ultralytics import YOLO


@register(name='yolov8')
class YOLOv8(NNOperator):
    def __init__(self, model="yolov8n.pt"):
        super().__init__()
        self._model = YOLO(model)

    def __call__(self, img: numpy.ndarray):
        results = self._model(img)[0]
        boxes = [list(map(int, xyxy)) for xyxy in result.boxes.xyxy]
        classes = [result.names[int(i)] for i in result.boxes.cls]
        scores = result.boxes.conf.tolist()
        return boxes, classes, scores


If you compare the YOLOv8 class with the YOLOv5 class, you will see that they are very similar, the difference is the usage of the models.

Then, we can also take the names `yolov8` operator to the pipeline, a similar pipeline as above, but changing the operator to YOLOv8.

In [ ]:
from towhee import pipe, ops, DataCollection

p = (
    pipe.input('path')
        .map('path', 'img', ops.image_decode.cv2_rgb())
        .map('img', ('box', 'class', 'score'), ops.yolov8(model="yolov8n.pt"))
        .map(('img', 'box'), 'object', ops.image_crop(clamp=True))
        .output('img', 'object', 'class')
)

In [ ]:
DataCollection(p("https://ultralytics.com/images/bus.jpg")).show()

Now we have completed the development of the YOLOv8 operator, as well as adding the operator to the pipeline, and the YOLOv8 model detects one more object, the stop sign, compared to the YOLOv5 model, so it is clear that YOLOv8 is a more complete detection than YOLOv5.

We can also develop the `__init__` and `__call__` methods according to how models such as YOLOv6 are used, for example, to enable different models.

## Set for training

We can also train the YOLOv8 operator, according to the method of [YOLOv8 training](https://github.com/ultralytics/ultralytics/blob/dce4efce48a05e028e6ec430045431c242e52484/docs/yolov5/tutorials/train_custom_data.md), first of all, you have to manually create a "dataset" directory in the current directory, and then you can use COCO dataset to train the YOLOv8 model, or you can replace it with your own dataset.

In [ ]:
import towhee

op = towhee.ops.yolov8().get_op()

In [ ]:
op._model.train(data="coco128.yaml", epochs=3)